In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import transformers
from transformers import (
    AdamW,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_linear_schedule_with_warmup)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoModel, BertTokenizerFast
from transformers import AutoTokenizer, DataCollatorWithPadding
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import pytorch_lightning as pl
import os.path
from torch.optim.lr_scheduler import ReduceLROnPlateau
from transformers import BertTokenizer, BertModel
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, DataCollatorWithPadding
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import wandb
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import TQDMProgressBar, ModelCheckpoint, EarlyStopping

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## **Data Preparation**

In [5]:
labelled_data = pd.read_csv('/Users/vladandreichuk/Desktop/practical-data-science-tutorial/data/Final-Dataset - finaldataset.csv')

In [26]:
test_data = pd.read_csv('/Users/vladandreichuk/Desktop/practical-data-science-tutorial/data/Draft-dataset - draftdataset.csv')

In [7]:
labelled_data = labelled_data[['study_title','Confidence']]

In [27]:
test_data = test_data[['study_title']]

In [28]:
test_data

,study_title
0,The psychological impact of quarantine and how...
1,"Global, regional, and national incidence, prev..."
2,A novel coronavirus outbreak of global health ...
3,COVID-19 and Italy: what next?
4,Structural racism and health inequities in the...
...,...
1237,Informing the Market: The Effect of Modern Inf...
1238,Winning by Losing: Evidence on the Long-run Ef...
1239,Employment and Wage Insurance within Firms: Wo...
1240,Why Does Fast Loan Growth Predict Poor Perform...


In [9]:
labelled_data['Confidence'] = labelled_data['Confidence'].round().astype(int)

/var/folders/sd/m36c35x53hb666q0qqt92wk40000gn/T/ipykernel_18429/1189677785.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labelled_data['Confidence'] = labelled_data['Confidence'].round().astype(int)


In [12]:
labelled_data.columns = ['body', 'label']

In [29]:
test_data.columns = [['body']]

In [39]:
# Remove duplicates from the 'body' column in both DataFrames
test_data = test_data.to_frame(name='body')
test_data = test_data.drop_duplicates().reset_index(drop=True)
labelled_data = labelled_data.drop_duplicates(subset=['body'])

# Find common rows based on the 'body' column
common_rows = test_data.merge(labelled_data, on='body', how='inner')

# Get the indices of the common rows in 'test_data'
common_indices = test_data.index[test_data['body'].isin(common_rows['body'])]

# Remove common rows from 'test_data'
test_data = test_data.drop(common_indices)

In [40]:
test_data

,body
0,The psychological impact of quarantine and how...
1,"Global, regional, and national incidence, prev..."
2,A novel coronavirus outbreak of global health ...
3,COVID-19 and Italy: what next?
4,Structural racism and health inequities in the...
...,...
1231,Informing the Market: The Effect of Modern Inf...
1232,Winning by Losing: Evidence on the Long-run Ef...
1233,Employment and Wage Insurance within Firms: Wo...
1234,Why Does Fast Loan Growth Predict Poor Perform...


In [64]:
labelled_data

,body,label
0,COVID-19: the gendered impacts of the outbreak,3
1,COVID-19: towards controlling of a pandemic,1
2,Prevention and treatment of low back pain: evi...,4
3,Effect of early tranexamic acid administration...,4
4,Offline: COVID-19 is not a pandemic,4
...,...,...
95,The role and dimensions of authenticity in her...,2
96,Impact of crisis events on Chinese outbound to...,4
97,Effects of the Booking.com rating system: Brin...,4
98,Spillover effects of economic globalization on...,4


In [66]:
labelled_data['label'] = labelled_data['label'] - 1

In [67]:
labelled_data['label'].value_counts()

label
1    35
3    30
2    21
0    12
4     2
Name: count, dtype: int64

In [68]:
# Split the data into training and validation sets
train_data, val_data = train_test_split(labelled_data, test_size=0.2, random_state=42)

## **Model**

### **BERT Model**

In [86]:
epoch = 0


class BERT_unfreeze(pl.LightningModule):  
    def __init__(self, weights,max_length, learning_rate=1e-5):
        super().__init__()
        self.save_hyperparameters()
        self.loss = nn.NLLLoss(weight=weights)
        self.max_length=max_length
        self.learning_rate = learning_rate
        self.training_step_loss = [] 
        self.validation_step_loss = []
        self.test_step_loss = []
        self.test_step_acc = []
        self.training_step_acc = []
        self.validation_step_acc = []

        self.bert = BertModel.from_pretrained("bert-base-uncased")

        #freeze the pretrained layers except 1-3
        for layer in self.bert.encoder.layer[:9]:
            for param in layer.parameters():
                param.requires_grad = False
       
        # dropout layer
        self.dropout = nn.Dropout(0.2)
       
         # relu activation function
        self.relu =  nn.ReLU()
 
         # dense layer 1
        self.fc1 = nn.Linear(768,512)
       
        # dense layer 2 (Output layer)
        self.fc2 = nn.Linear(512,5)
 
        #softmax activation function
        self.softmax = nn.LogSoftmax(dim=1)
 
    #define the forward pass
    def forward(self, sent_id, mask):
 
        #pass the inputs to the model
        _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False)
       
        x = self.fc1(cls_hs)
 
        x = self.relu(x)
 
        x = self.dropout(x)
 
        # output layer
        x = self.fc2(x)
       
        # apply softmax activation
        x = self.softmax(x)
 
        return x

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.learning_rate)
        scheduler = {'scheduler':ReduceLROnPlateau(optimizer,mode='min',factor=0.5,patience=3,verbose=1),
'monitor':'val_loss'}
        return {'optimizer': optimizer, 'lr_scheduler': scheduler}

    def export_metrics(self, acc, loss, step):

        metrics_filename = f"/Users/vladandreichuk/Desktop/practical-data-science-tutorial/models/BERT_metrics/max_l{self.max_length}_lr{self.learning_rate}_{step}_metrics.csv"

        metrics_df = pd.DataFrame([[epoch, acc,float(loss)]], columns = ['epoch', 'acc', 'loss'])

        if os.path.isfile(metrics_filename):

            metrics_in_data = pd.read_csv(metrics_filename)
            metrics_out_data = pd.concat([metrics_in_data, metrics_df])
            metrics_out_data.to_csv(metrics_filename, index = False)

        else:

            metrics_df.to_csv(metrics_filename, index = False)

        return

    # function to train the model
    def training_step(self, train_batch, batch_idx):
        sent_id, mask, labels= train_batch

        # get model predictions for the current batch
        preds = self(sent_id, mask)

        # compute the loss between actual and predicted values
        loss = self.loss(preds, labels)

        acc = torch.sum(labels == torch.argmax(preds, dim=1)).item() / len(labels)

        metrics = {"train_acc": acc, "loss": loss}

        self.training_step_loss.append(loss) 
        self.training_step_acc.append(acc) 

        return metrics


    def on_train_epoch_end(self):
        step = "train"
        loss = torch.stack(self.training_step_loss).mean()
        acc = sum(self.training_step_acc) / len(self.training_step_acc)
        self.export_metrics(acc, loss, step)
        self.training_step_loss.clear() 
        self.training_step_acc.clear() 

    def validation_step(self, val_batch, batch_idx):
        
        sent_id, mask, labels = val_batch
        preds = self(sent_id, mask)
        loss, acc = self._shared_eval_step(val_batch, batch_idx)
        metrics = {"val_acc": acc, "val_loss": loss}
        self.log_dict(metrics)
        self.validation_step_loss.append(loss)
        self.validation_step_acc.append(acc)

        return metrics
    

    def on_validation_epoch_end(self):
        global epoch
        epoch = epoch + 1

        step = "val"
        loss = torch.stack(self.validation_step_loss).mean()
        acc = sum(self.validation_step_acc) / len(self.validation_step_acc)
        self.export_metrics(acc, loss, step)
        self.validation_step_loss.clear() 
        self.validation_step_acc.clear() 

    def test_step(self, test_batch, batch_idx):

        sent_id, mask, labels= test_batch
        preds = self(sent_id, mask)
        loss, acc = self._shared_eval_step(test_batch, batch_idx)
        metrics = {"test_acc": acc, "test_loss": loss}
        self.export_metrics(acc,loss,'test')
        self.log_dict(metrics)
        self.test_step_loss.append(loss)

        return metrics

    def predict_step(self, predict_batch, batch_idx):
        sent_id, mask = predict_batch

        return self(sent_id, mask)

    def _shared_eval_step(self, batch, batch_idx):
        sent_id, mask, labels= batch

        # model predictions
        preds = self(sent_id, mask)

        # compute the validation loss between actual and predicted values
        loss = self.loss(preds, labels)

        # compute accuracy between actual and predicted values
        acc = torch.sum(labels == torch.argmax(preds, dim=1)).item() / len(labels)

        return loss, acc

### **Data Loader**

In [88]:
class DataModule(pl.LightningDataModule):
    def __init__(self, batch_size=32, max_length=256, test_nrows=None):
        super().__init__()
        self.batch_size = batch_size
        self.max_length = max_length
        self.test_nrows = test_nrows
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def tokenize(self, df):
        tokens = self.tokenizer.__call__(
                df['body'].tolist(),
                padding = True, max_length=self.max_length,
                truncation = True)
               
        # convert the integer sequences to tensors.
        seq = torch.tensor(tokens['input_ids'])
        mask = torch.tensor(tokens['attention_mask'])
        if 'label' in df.columns:
            y = torch.tensor(df['label'].tolist())
            return TensorDataset(seq, mask, y)
        else:
            return TensorDataset(seq, mask)

    def setup(self, stage):
        if stage == "weight_calc":
            df_train = train_data
            class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(df_train['label']), y=df_train['label'])
            self.weights = torch.tensor(class_weights, dtype=torch.float)

        if stage == "fit":
            df_train =  train_data
            df_val =  val_data

            self.train_data = self.tokenize(df_train)
            self.val_data = self.tokenize(df_val)

        if stage == "test" or stage == "predict":
            df_test = test_data
            self.test_data = self.tokenize(df_test)

    def get_weights(self):
        return self.weights
       
    def train_dataloader(self):
        return DataLoader(self.train_data, drop_last=True, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val_data, drop_last=True, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_data, drop_last=True, batch_size=self.batch_size)

    def predict_dataloader(self):
        return DataLoader(self.test_data, drop_last=True, batch_size=self.batch_size)

### **Model Training**

In [89]:
def train_predict():
    dm = DataModule(batch_size=5, max_length=512,test_nrows=None)
    dm.setup(stage="weight_calc")

    model = BERT_unfreeze(weights=dm.get_weights(), max_length=512, learning_rate=1e-4)

    trainer = pl.Trainer(default_root_dir='/Users/vladandreichuk/Desktop/practical-data-science-tutorial/models/BERT_metrics/', max_epochs=20, log_every_n_steps=10, gradient_clip_val=1.0, accelerator='auto', devices='1',
                         callbacks=[EarlyStopping(monitor="val_loss", min_delta = 0.0001, patience=5, mode="min"),ModelCheckpoint(save_top_k=1, save_last=False, monitor="val_loss"),TQDMProgressBar(refresh_rate=10)])

    trainer.fit(model, dm)

    # Load the best checkpoint for prediction
    best_model_checkpoint = trainer.checkpoint_callback.best_model_path
    best_model = BERT_unfreeze.load_from_checkpoint(best_model_checkpoint)
    predictions = trainer.predict(best_model,dm)

    return predictions

predictions = train_predict()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

  | Name    | Type       | Params
---------------------------------------
0 | loss    | NLLLoss    | 0     
1 | bert    | BertModel  | 109 M 
2 | dropout | Dropout    | 0     
3 | relu    | ReLU       | 0     
4 | fc1     | Linear     | 393 K 
5 | fc2     | Linear     | 2.6 K 
6 | softmax | LogSoftmax | 0     
---------------------------------------
46.1 M    Trainable params
63.8 M    Non-trainable params
109 M     Total params
439.514   Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 9: 100%|██████████| 16/16 [00:16<00:00,  0.99it/s, v_num=10]


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 228/228 [00:48<00:00,  4.71it/s]


## **Predictions**

In [111]:
predicted_labels = []
for i in range(len(predictions)):
    predicted_labels_batch = torch.argmax(predictions[i], dim=1)
    predicted_labels_batch_list = predicted_labels_batch.tolist()
    predicted_labels.extend(predicted_labels_batch_list)

In [114]:
predicted_labels

[3,
 3,
 0,
 0,
 1,
 1,
 3,
 0,
 3,
 1,
 3,
 1,
 0,
 1,
 3,
 3,
 3,
 3,
 1,
 1,
 3,
 3,
 3,
 3,
 1,
 1,
 3,
 3,
 3,
 3,
 2,
 3,
 2,
 3,
 2,
 1,
 0,
 3,
 1,
 0,
 3,
 1,
 1,
 0,
 1,
 1,
 3,
 3,
 1,
 3,
 0,
 3,
 1,
 3,
 0,
 3,
 3,
 3,
 0,
 3,
 3,
 2,
 0,
 1,
 1,
 1,
 1,
 3,
 1,
 3,
 3,
 3,
 3,
 1,
 3,
 3,
 3,
 1,
 3,
 3,
 3,
 3,
 1,
 1,
 1,
 1,
 3,
 3,
 0,
 3,
 3,
 1,
 3,
 1,
 3,
 3,
 3,
 1,
 3,
 1,
 1,
 1,
 3,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 3,
 3,
 3,
 3,
 3,
 3,
 1,
 1,
 1,
 3,
 3,
 1,
 1,
 1,
 3,
 3,
 3,
 1,
 0,
 3,
 1,
 0,
 1,
 0,
 1,
 3,
 1,
 1,
 1,
 1,
 3,
 1,
 3,
 3,
 1,
 1,
 3,
 1,
 3,
 1,
 2,
 3,
 3,
 3,
 1,
 3,
 1,
 3,
 3,
 3,
 1,
 3,
 0,
 1,
 1,
 3,
 1,
 2,
 3,
 2,
 2,
 3,
 2,
 3,
 3,
 3,
 1,
 0,
 1,
 3,
 1,
 3,
 3,
 3,
 0,
 1,
 1,
 1,
 0,
 0,
 3,
 3,
 0,
 1,
 0,
 3,
 3,
 1,
 3,
 1,
 1,
 1,
 3,
 3,
 3,
 1,
 3,
 3,
 3,
 1,
 3,
 3,
 3,
 1,
 3,
 3,
 0,
 3,
 1,
 3,
 3,
 3,
 0,
 3,
 3,
 1,
 3,
 3,
 3,
 3,
 3,
 3,
 1,
 3,
 3,
 3,
 3,
 0,
 3,
 3,
 1,
 1,
 3,
 3,
 3,
 3,
 1,
 3,
 3,


In [115]:
test_data['BERT_label'] = predicted_labels

In [116]:
test_data

,body,BERT_label
0,The psychological impact of quarantine and how...,3
1,"Global, regional, and national incidence, prev...",3
2,A novel coronavirus outbreak of global health ...,0
3,COVID-19 and Italy: what next?,0
4,Structural racism and health inequities in the...,1
...,...,...
1231,Informing the Market: The Effect of Modern Inf...,2
1232,Winning by Losing: Evidence on the Long-run Ef...,3
1233,Employment and Wage Insurance within Firms: Wo...,1
1234,Why Does Fast Loan Growth Predict Poor Perform...,2


In [117]:
test_data.to_csv('/Users/vladandreichuk/Desktop/practical-data-science-tutorial/data/Full_Dataset_Labelled_BERT', index=False) 